In [15]:
import json
import os
import shopify
import pickle
from dotenv import load_dotenv

In [16]:

load_dotenv()
session = shopify.Session(os.environ["SHOPIFY_SHOP_URL"], os.environ["SHOPIFY_API_VERSION"], os.environ["SHOPIFY_ACCESS_TOKEN"])
shopify.ShopifyResource.activate_session(session)

In [ ]:
description = "Find user id by email. If the user is not found, the function will return an error message."
slots = [
    {
        "name": "email",
        "type": "string",
        "description": "The email of the user, such as 'something@example.com'.",
        "prompt": "In order to proceed, please provide the email for identity verification.",
        "required": True,
    }
]
outputs = [
    {
        "name": "user_id",
        "type": "string",
        "description": "The user id of the user. such as 'gid://shopify/Customer/13573257450893'.",
    }
]

USER_NOT_FOUND_ERROR = "error: user not found"
MULTIPLE_USERS_SAME_EMAIL_ERROR = "error: there are multiple users with the same email"

def find_user_id_by_email(email: str) -> str:
    user_id = ""
    try:
        response = shopify.GraphQL().execute(f"""
            {{
                customers (first: 10, query: "email:{email}") {{
                    edges {{
                        node {{
                            id
                        }}
                    }}
                }}
            }}
            """)
        nodes = json.loads(response)["data"]["customers"]["edges"]
        if len(nodes) == 1:
            user_id = nodes[0]["node"]["id"]
            return user_id
        else:
            return MULTIPLE_USERS_SAME_EMAIL_ERROR
    except Exception as e:
        return USER_NOT_FOUND_ERROR

In [79]:
find_user_id_by_email("luyunan0404@gmail.com")

'gid://shopify/Customer/7450970259569'

In [ ]:
description = "Search collections by string query. If no collections are found, the function will return an error message."
slots = [
    {
        "name": "query",
        "type": "string",
        "description": "The string query to search collections, such as 'Hats'. If query is empty string, it returns all collections. If there is only 1 product, return in list with single item.",
        "prompt": "In order to proceed, please provide a query for the collections search.",
        "required": False,
    }
]
outputs = [
    {
        "name": "user_id",
        "type": "string",
        "description": "A list of up to 10 collections that satisfies the query. Such as ",
    }
]
COLLECTION_SEARCH_ERROR = "error: catalog search failed"
COLLECTION_NOT_FOUND_ERROR = "no collections found"

def search_collections(query: str) -> str:
    user_id = ""
    
    try:
        response = shopify.GraphQL().execute(f"""
            {{
                collections (first: 10, query: "{query}") {{
                    edges {{
                        node {{
                            description
                            title
                            id
                            productsCount {{
                                count
                            }}
                        }}  
                    }}
                    nodes {{
                        description
                        title
                        id
                        productsCount {{
                            count
                        }}
                    }}  
                }}
            }}
            """)
        display(json.loads(response))
        nodes = json.loads(response)["data"]["collections"]["edges"]
        display()
        if len(nodes):
            return [c['node'] for c in nodes]
        else:
            return COLLECTION_NOT_FOUND_ERROR
    
    except Exception as e:
        return COLLECTION_SEARCH_ERROR

In [116]:
get_collections("bedding")

{'data': {'collections': {'edges': [{'node': {'description': '',
      'title': 'Beddings and Pillows',
      'id': 'gid://shopify/Collection/284023292017',
      'productsCount': {'count': 6}}},
    {'node': {'description': '',
      'title': 'Bedding',
      'id': 'gid://shopify/Collection/284607709297',
      'productsCount': {'count': 4}}}],
   'nodes': [{'description': '',
     'title': 'Beddings and Pillows',
     'id': 'gid://shopify/Collection/284023292017',
     'productsCount': {'count': 6}},
    {'description': '',
     'title': 'Bedding',
     'id': 'gid://shopify/Collection/284607709297',
     'productsCount': {'count': 4}}]}},
 'extensions': {'cost': {'requestedQueryCost': 18,
   'actualQueryCost': 6,
   'throttleStatus': {'maximumAvailable': 2000.0,
    'currentlyAvailable': 1994,
    'restoreRate': 100.0}}}}

[{'description': '',
  'title': 'Beddings and Pillows',
  'id': 'gid://shopify/Collection/284023292017',
  'productsCount': {'count': 6}},
 {'description': '',
  'title': 'Bedding',
  'id': 'gid://shopify/Collection/284607709297',
  'productsCount': {'count': 4}}]

'gid://shopify/Collection/284023292017'

In [47]:
pickle.load(open("/Users/isaac/programming/articulateai/AgentOrg/examples/shopify/chunked_documents.pkl", 'rb'))

[Document(metadata={'title': 'mmermind fleece blankets and throws Adult Thick Warm winter Blankets Home Super Soft duvet luxury solid Blankets On twin Bedding'}, page_content='Brand Name: mmermindOrigin: CN(Origin)Material: 100% PolyesterFeature: Anti-PillingPattern Type: SolidSeason: WinterType: Coral Fleece FabricGrade: Grade AModel Number: SG52Pattern: FlockedStyle: ModernSet Type: NoColor Fastness (Grade): 4Technics: NonwovenShape: SquareUse: HospitalUse: HomeUse: HOTELWeight: 1000gWash Style: Mechanical WashAge Group: Adultsfleece blankets: fleece blanketswinter Blankets: winter Blankets'),
 Document(metadata={'title': 'Winter Flannel Blanket Solid Color Plaid Coral Blanket Fleece Bedspread For Bed Sofa Thicken Plush Blanket Thin Quilt Home Decor'}, page_content='Origin: CN(Origin)Material: Microfiber FabricFeature: HeatedFeature: Anti-PillingFeature: PortablePattern Type: SolidSeason: WinterType: Flannel FabricGrade: Grade AModel Number: MT005Pattern: Plain DyedStyle: ModernSet T

In [11]:
pickle.load(open("/Users/isaac/programming/articulateai/AgentOrg/examples/shopify/documents.pkl", 'rb'))

[Document(metadata={'title': 'mmermind fleece blankets and throws Adult Thick Warm winter Blankets Home Super Soft duvet luxury solid Blankets On twin Bedding'}, page_content='Brand Name: mmermindOrigin: CN(Origin)Material: 100% PolyesterFeature: Anti-PillingPattern Type: SolidSeason: WinterType: Coral Fleece FabricGrade: Grade AModel Number: SG52Pattern: FlockedStyle: ModernSet Type: NoColor Fastness (Grade): 4Technics: NonwovenShape: SquareUse: HospitalUse: HomeUse: HOTELWeight: 1000gWash Style: Mechanical WashAge Group: Adultsfleece blankets: fleece blanketswinter Blankets: winter Blankets'),
 Document(metadata={'title': 'Winter Flannel Blanket Solid Color Plaid Coral Blanket Fleece Bedspread For Bed Sofa Thicken Plush Blanket Thin Quilt Home Decor'}, page_content='Origin: CN(Origin)Material: Microfiber FabricFeature: HeatedFeature: Anti-PillingFeature: PortablePattern Type: SolidSeason: WinterType: Flannel FabricGrade: Grade AModel Number: MT005Pattern: Plain DyedStyle: ModernSet T